In [25]:
import requests
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [43]:
data2 = open('/home/chirag/Downloads/Stsbenchmark/stsbenchmark/sts-test.csv')
d2 = data2.readlines()
data = d2
print(len(data))

1379


In [27]:
df = []
for i in range(len(data)):
    df.append(data[i].split('\t'))
score = []
sentence1 = []
sentence2 = []
for j in range(len(df)):
    score.append(df[j][4])
    sentence1.append(df[j][5])
    b = df[j][6]
    sentence2.append(b.rstrip("\n"))

In [28]:
dict = list(zip(sentence1, sentence2, score))
df = pd.DataFrame(dict, columns = ['sentence1', 'sentence2', 'score'])

In [29]:
scaler = MinMaxScaler()
df['score'] = scaler.fit_transform(df['score'].values.reshape(-1,1))

In [30]:
df.head()

,sentence1,sentence2,score
0,A girl is styling her hair.,A girl is brushing her hair.,0.50
1,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.,0.72
2,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.,1.00
3,A man is cutting up a cucumber.,A man is slicing a cucumber.,0.84
4,A man is playing a harp.,A man is playing a keyboard.,0.30


In [31]:
URL = 'https://salesken-analysis-249709.appspot.com/sentence_similarity_roberta'

In [32]:
rs = []
rob = []
for i in range(len(df)):
    data = {'sentence1': "[\"" + df.sentence1[i] + "\"]",
        'sentence2': "[\"" + df.sentence2[i] + "\"]"}
    r = requests.post(url = URL, data = data) 
#     print(r.json())
    rs.append(r.json())
    rob.append(rs[i][0][0]["roberta_score"])
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [34]:
df["roberta_app"] = rob

In [35]:
import numpy as np
x=np.array(df["roberta_app"])
df["roberta_app_engine"] =np.asfarray(x,float)
df.drop(columns=['roberta_app'])

In [36]:
df.head()

,sentence1,sentence2,score,roberta_app,roberta_app_engine
0,A girl is styling her hair.,A girl is brushing her hair.,0.50,0.5791594982147217,0.579159
1,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.,0.72,0.6934813261032104,0.693481
2,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.,1.00,0.9417304396629333,0.941730
3,A man is cutting up a cucumber.,A man is slicing a cucumber.,0.84,0.8025397062301636,0.802540
4,A man is playing a harp.,A man is playing a keyboard.,0.30,0.34475386142730713,0.344754


In [38]:
scaler = MinMaxScaler()
df['roberta_app_engine'] = scaler.fit_transform(df['roberta_app_engine'].values.reshape(-1,1))

In [42]:
import psycopg2
try:
    connection = psycopg2.connect(user="postgres",
                                  password="cx6ac54nmgGtLD1y",
                                  host="35.200.234.61",
                                  port="5432",
                                  database="sales")
    for i in range(len(df)):
        cursor = connection.cursor()
        postgres_insert_query = """ UPDATE public.sts_benchmark SET roberta_score = %s, source_='sts_test' WHERE sentence1 = %s and sentence2 = %s and score=%s"""
        record_to_insert = (df.roberta_app_engine[i], df.sentence1[i], df.sentence2[i], df.score[i])
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        count = cursor.rowcount
        print (i, "Record inserted successfully into mobile table")
except (Exception, psycopg2.Error) as error :
    if(connection):
        print("Failed to insert record into mobile table", error)
finally:
    #closing database connection.
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

0 Record inserted successfully into mobile table
1 Record inserted successfully into mobile table
2 Record inserted successfully into mobile table
3 Record inserted successfully into mobile table
4 Record inserted successfully into mobile table
5 Record inserted successfully into mobile table
6 Record inserted successfully into mobile table
7 Record inserted successfully into mobile table
8 Record inserted successfully into mobile table
9 Record inserted successfully into mobile table
10 Record inserted successfully into mobile table
11 Record inserted successfully into mobile table
12 Record inserted successfully into mobile table
13 Record inserted successfully into mobile table
14 Record inserted successfully into mobile table
15 Record inserted successfully into mobile table
16 Record inserted successfully into mobile table
17 Record inserted successfully into mobile table
18 Record inserted successfully into mobile table
19 Record inserted successfully into mobile table
20 Record 

164 Record inserted successfully into mobile table
165 Record inserted successfully into mobile table
166 Record inserted successfully into mobile table
167 Record inserted successfully into mobile table
168 Record inserted successfully into mobile table
169 Record inserted successfully into mobile table
170 Record inserted successfully into mobile table
171 Record inserted successfully into mobile table
172 Record inserted successfully into mobile table
173 Record inserted successfully into mobile table
174 Record inserted successfully into mobile table
175 Record inserted successfully into mobile table
176 Record inserted successfully into mobile table
177 Record inserted successfully into mobile table
178 Record inserted successfully into mobile table
179 Record inserted successfully into mobile table
180 Record inserted successfully into mobile table
181 Record inserted successfully into mobile table
182 Record inserted successfully into mobile table
183 Record inserted successfull

325 Record inserted successfully into mobile table
326 Record inserted successfully into mobile table
327 Record inserted successfully into mobile table
328 Record inserted successfully into mobile table
329 Record inserted successfully into mobile table
330 Record inserted successfully into mobile table
331 Record inserted successfully into mobile table
332 Record inserted successfully into mobile table
333 Record inserted successfully into mobile table
334 Record inserted successfully into mobile table
335 Record inserted successfully into mobile table
336 Record inserted successfully into mobile table
337 Record inserted successfully into mobile table
338 Record inserted successfully into mobile table
339 Record inserted successfully into mobile table
340 Record inserted successfully into mobile table
341 Record inserted successfully into mobile table
342 Record inserted successfully into mobile table
343 Record inserted successfully into mobile table
344 Record inserted successfull

487 Record inserted successfully into mobile table
488 Record inserted successfully into mobile table
489 Record inserted successfully into mobile table
490 Record inserted successfully into mobile table
491 Record inserted successfully into mobile table
492 Record inserted successfully into mobile table
493 Record inserted successfully into mobile table
494 Record inserted successfully into mobile table
495 Record inserted successfully into mobile table
496 Record inserted successfully into mobile table
497 Record inserted successfully into mobile table
498 Record inserted successfully into mobile table
499 Record inserted successfully into mobile table
500 Record inserted successfully into mobile table
501 Record inserted successfully into mobile table
502 Record inserted successfully into mobile table
503 Record inserted successfully into mobile table
504 Record inserted successfully into mobile table
505 Record inserted successfully into mobile table
506 Record inserted successfull

649 Record inserted successfully into mobile table
650 Record inserted successfully into mobile table
651 Record inserted successfully into mobile table
652 Record inserted successfully into mobile table
653 Record inserted successfully into mobile table
654 Record inserted successfully into mobile table
655 Record inserted successfully into mobile table
656 Record inserted successfully into mobile table
657 Record inserted successfully into mobile table
658 Record inserted successfully into mobile table
659 Record inserted successfully into mobile table
660 Record inserted successfully into mobile table
661 Record inserted successfully into mobile table
662 Record inserted successfully into mobile table
663 Record inserted successfully into mobile table
664 Record inserted successfully into mobile table
665 Record inserted successfully into mobile table
666 Record inserted successfully into mobile table
667 Record inserted successfully into mobile table
668 Record inserted successfull

811 Record inserted successfully into mobile table
812 Record inserted successfully into mobile table
813 Record inserted successfully into mobile table
814 Record inserted successfully into mobile table
815 Record inserted successfully into mobile table
816 Record inserted successfully into mobile table
817 Record inserted successfully into mobile table
818 Record inserted successfully into mobile table
819 Record inserted successfully into mobile table
820 Record inserted successfully into mobile table
821 Record inserted successfully into mobile table
822 Record inserted successfully into mobile table
823 Record inserted successfully into mobile table
824 Record inserted successfully into mobile table
825 Record inserted successfully into mobile table
826 Record inserted successfully into mobile table
827 Record inserted successfully into mobile table
828 Record inserted successfully into mobile table
829 Record inserted successfully into mobile table
830 Record inserted successfull

973 Record inserted successfully into mobile table
974 Record inserted successfully into mobile table
975 Record inserted successfully into mobile table
976 Record inserted successfully into mobile table
977 Record inserted successfully into mobile table
978 Record inserted successfully into mobile table
979 Record inserted successfully into mobile table
980 Record inserted successfully into mobile table
981 Record inserted successfully into mobile table
982 Record inserted successfully into mobile table
983 Record inserted successfully into mobile table
984 Record inserted successfully into mobile table
985 Record inserted successfully into mobile table
986 Record inserted successfully into mobile table
987 Record inserted successfully into mobile table
988 Record inserted successfully into mobile table
989 Record inserted successfully into mobile table
990 Record inserted successfully into mobile table
991 Record inserted successfully into mobile table
992 Record inserted successfull

1133 Record inserted successfully into mobile table
1134 Record inserted successfully into mobile table
1135 Record inserted successfully into mobile table
1136 Record inserted successfully into mobile table
1137 Record inserted successfully into mobile table
1138 Record inserted successfully into mobile table
1139 Record inserted successfully into mobile table
1140 Record inserted successfully into mobile table
1141 Record inserted successfully into mobile table
1142 Record inserted successfully into mobile table
1143 Record inserted successfully into mobile table
1144 Record inserted successfully into mobile table
1145 Record inserted successfully into mobile table
1146 Record inserted successfully into mobile table
1147 Record inserted successfully into mobile table
1148 Record inserted successfully into mobile table
1149 Record inserted successfully into mobile table
1150 Record inserted successfully into mobile table
1151 Record inserted successfully into mobile table
1152 Record 

1291 Record inserted successfully into mobile table
1292 Record inserted successfully into mobile table
1293 Record inserted successfully into mobile table
1294 Record inserted successfully into mobile table
1295 Record inserted successfully into mobile table
1296 Record inserted successfully into mobile table
1297 Record inserted successfully into mobile table
1298 Record inserted successfully into mobile table
1299 Record inserted successfully into mobile table
1300 Record inserted successfully into mobile table
1301 Record inserted successfully into mobile table
1302 Record inserted successfully into mobile table
1303 Record inserted successfully into mobile table
1304 Record inserted successfully into mobile table
1305 Record inserted successfully into mobile table
1306 Record inserted successfully into mobile table
1307 Record inserted successfully into mobile table
1308 Record inserted successfully into mobile table
1309 Record inserted successfully into mobile table
1310 Record 

In [ ]:
y_true = [1,0,0,0,1,1]
y_pred = [1,1,0,1,0,1]
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
